In [1]:
import pandas as pd
from pathlib import Path

path = Path.home() / '.data' / 'asl-signs'

In [3]:
sign_meta = (pd.read_csv(path / 'train_with_meta.csv', dtype=dict(sequence_id=str))
             .set_index('sequence_id')
            )

COORD_COLS = ['x', 'y', 'z']
LANDMARK_TYPES = ['left_hand', 'pose', 'right_hand']

In [26]:
# sequence_id = '1951401018'
sequence_id = '115719649'
meta = sign_meta.loc[sequence_id, :]
meta

idx                                                        2044_115719649
path                          train_landmark_files/2044/115719649.parquet
participant_id                                                       2044
sign                                                                 blow
cnt_partial_nulls                                                     0.0
cnt_partial_nulls_by_frame                                            0.0
total_frames                                                          6.0
face                                                                  6.0
left_hand                                                             0.0
pose                                                                  6.0
right_hand                                                            3.0
Name: 115719649, dtype: object

In [27]:
landmarks = pd.read_parquet(path / meta['path'])
landmarks = landmarks.loc[landmarks.type.isin(LANDMARK_TYPES), ]

In [28]:
mask = ((landmarks.type == 'right_hand') & 
        (landmarks.landmark_index == 0) #&
        #(landmarks.frame.between(29, 31))
       )
landmarks.loc[mask, COORD_COLS].multiply(100).round(2)

,x,y,z
522,NaN,NaN,NaN
1065,NaN,NaN,NaN
1608,NaN,NaN,NaN
2151,26.47,64.99,0.0
2694,28.61,62.25,0.0
3237,28.48,62.06,0.0


In [29]:
inter_dims = ['type', 'landmark_index']
landmarks = (landmarks.sort_values(['type', 'landmark_index', 'frame'])
             .assign(x=lambda x: x.groupby(inter_dims).x.apply(lambda g: g.interpolate()),
                     y=lambda x: x.groupby(inter_dims).y.apply(lambda g: g.interpolate()),
                     z=lambda x: x.groupby(inter_dims).z.apply(lambda g: g.interpolate()),
              )
            )

In [30]:
mask = ((landmarks.type == 'right_hand') & 
        (landmarks.landmark_index == 0) #&
        #(landmarks.frame.between(29, 31))
       )
landmarks.loc[mask, COORD_COLS].multiply(100).round(2)

,x,y,z
522,NaN,NaN,NaN
1065,NaN,NaN,NaN
1608,NaN,NaN,NaN
2151,26.47,64.99,0.0
2694,28.61,62.25,0.0
3237,28.48,62.06,0.0
